In [1]:
# Get the required packages and libraries
import os
import random
import shutil
import cv2
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.models import load_model

import matplotlib.image as mpimg
from sklearn.preprocessing import LabelBinarizer

Using TensorFlow backend.


In [5]:
# Declaration of variables

proportion = 0.1
BATCH_SIZE = 20
SIZE = 150
DEFAULT_IMAGE_SIZE = tuple((150,150))
EPOCH = 5

In [ ]:
# # To take a proportion of whole data
# # Get train and val data
# data_train = os.listdir('D:/POST_S1/BANGKIT/TomatoLeaf/tomato/train_lite/')
# data_val = os.listdir('D:/POST_S1/BANGKIT/TomatoLeaf/tomato/test_lite/')

# # Get a proportion of whole data
# for folder in data_train:
#     i = 0
#     for data in os.listdir('tomato/val/'+folder):
#         if(i < proportion * len(os.listdir('tomato/val/'+folder))):
#             shutil.copyfile('tomato/val/'+folder+'/'+data, 'tomato/test_lite/'+folder+'/'+data)
#             i += 1

In [3]:
# Define the base directory (Change according to your data)
base_dir = 'D:/POST_S1/BANGKIT/TomatoLeaf/tomato/'

train_dir = base_dir + 'train_lite/'
test_dir = base_dir + 'test_lite/'

In [16]:
# Make ImageGenerator object as argument of fit function

# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( 
                    rescale = 1.0/255. ,
                    rotation_range=40,
                    width_shift_range=0.4,
                    height_shift_range=0.4,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    fill_mode='nearest'
)

test_datagen  = ImageDataGenerator( rescale = 1.0/255. )


train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical',
                                                    target_size=DEFAULT_IMAGE_SIZE)

test_generator =  test_datagen.flow_from_directory(test_dir,
                                                         batch_size=BATCH_SIZE,
                                                         class_mode  = 'categorical',
                                                         target_size = DEFAULT_IMAGE_SIZE)

Found 1000 images belonging to 10 classes.
Found 100 images belonging to 10 classes.


In [17]:
# Define the layers of model

#chanDim = -1
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', input_shape=(SIZE, SIZE, 3)),
    tf.keras.layers.BatchNormalization(axis=-1),
    tf.keras.layers.MaxPooling2D(3,3),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(axis=-1),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(axis=-1),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'), 
    tf.keras.layers.BatchNormalization(axis=-1),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'), 
    tf.keras.layers.BatchNormalization(axis=-1),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(), 
    # 512 neuron hidden layer
    tf.keras.layers.Dense(1024, activation='relu'), 
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('cats') and 1 for the other ('dogs')
    tf.keras.layers.Dense(10, activation='softmax')  

])

In [ ]:
# VGG Architecture
model = Sequential()
model.add(Conv2D(input_shape=(SIZE,SIZE,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=2, activation="softmax"))

In [18]:
# Show the architecture of layers
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 148, 148, 128)     3584      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 49, 49, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 47, 47, 128)       147584    
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 45, 45, 128)       147584    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 22, 22, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 20, 20, 128)       147584    
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 18, 18, 128)      

In [19]:
# Compile the model with optimizer
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

In [20]:
# Train data with defined model

history = model.fit(train_generator,
                    validation_data=test_generator,
                    steps_per_epoch=len(train_generator),
                    validation_steps =len(test_generator),
                    epochs=EPOCH)

Epoch 1/5
50/50 [==============================] - 126s 3s/step - loss: 2.3198 - accuracy: 0.0930 - val_loss: 2.2995 - val_accuracy: 0.1400
Epoch 2/5
50/50 [==============================] - 101s 2s/step - loss: 2.2442 - accuracy: 0.1540 - val_loss: 2.1846 - val_accuracy: 0.1700
Epoch 3/5
50/50 [==============================] - 81s 2s/step - loss: 2.1692 - accuracy: 0.1830 - val_loss: 1.9216 - val_accuracy: 0.3300
Epoch 4/5
50/50 [==============================] - 82s 2s/step - loss: 2.0884 - accuracy: 0.2090 - val_loss: 2.1170 - val_accuracy: 0.2100
Epoch 5/5
50/50 [==============================] - 82s 2s/step - loss: 1.9501 - accuracy: 0.2500 - val_loss: 1.9652 - val_accuracy: 0.2900


In [ ]:
# Plot the training result

fig, ax = plt.subplots(1,2,figsize=(18,3))
ax[0].plot(history.history["accuracy"])
ax[0].plot(history.history["val_accuracy"])
ax[0].set_title("Train and Validation Accuracy")
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Accuracy")
ax[0].legend(['Train', 'Validation'], loc='upper right')

ax[1].plot(history.history["loss"])
ax[1].plot(history.history["val_loss"])
ax[1].set_title("Train and Validation Loss")
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Loss")
ax[1].legend(["Train", "Validation"], loc="upper right")
plt.savefig(datetime.now().strftime("%m%d%Y_%H%M%S") + '.png')
plt.show()

In [11]:
# Save trained model into .h5 format

model.save('model1.h5')

In [12]:
# Predict the image using defined model

img=image.load_img('sehat.jpg', target_size=(150, 150))
x=image.img_to_array(img)
x=np.expand_dims(x, axis=0)
images = np.vstack([x])

model.predict(images, batch_size=BATCH_SIZE)

array([[0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        1.000000e+00, 4.679783e-21]], dtype=float32)

In [ ]:
# Install Tensorflowjs
#!pip install tensorflowjs

In [15]:
# Convert .h5 model into model.json
!tensorflowjs_converter --input_format keras model1.h5 model1